In [1]:
import os, sys
import rasterio
from rasterio.mask import mask
import numpy as np
from matplotlib import pyplot as plt
import glob
from xml.dom import minidom
import fiona
import scipy.misc as misc

%matplotlib inline

In [2]:
# function to return collection dates from Planet MultiSpectral image metadata
def getDates_planet(xml_file):

    xmldoc = minidom.parse(xml_file)
    date_node = xmldoc.getElementsByTagName("eop:acquisitionDate")
    date_str = str(date_node[0].firstChild.nodeValue)
    acq_date_yyyymmdd = date_str.split('T')[0]        
    
    return acq_date_yyyymmdd

# function to return correction coefficients from Planet MultiSpectral image metadata
def getCorrCoefs_planet(xml_file):

    xmldoc = minidom.parse(xml_file)
    nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

    # XML parser refers to bands by numbers 1-4
    coeffs = {}
    for node in nodes:
        bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if bn in ['1', '2', '3', '4']:
            i = int(bn)
            value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
            coeffs[i] = float(value)
            
    return coeffs

# function to extract image patch
def getImagePatch_planet(imfile, poly):
    
    try:
        with rasterio.open(imfile, 'r') as src:
            arr,_ = mask(src, [poly], crop=True)

        if arr.sum() != 0.:
            return arr
        else:
            return 0
        
    except:
        return 0

# function return geometries as geoJSON
def getGeometries_planet(shpfile):
    
    with fiona.open(shpfile, "r") as shapefile:
        geoms = [feature["geometry"] for feature in shapefile]
        
    return geoms


In [3]:
home_dir = r"C:\Projects\RD\planet\sample_order"

all_ims = glob.glob('{}/*/*.tif'.format(home_dir))
all_ms_ims = [im for im in all_ims if "MS.tif" in im]
all_sr_ims = [im for im in all_ims if "SR" in im]
all_qa = [im for im in all_ims if "DN_UDM" in im]
all_xml = glob.glob('{}/*/*.xml'.format(home_dir))

In [4]:
c = getCorrCoefs_planet(all_xml[0])
print(c[4])
print(c)


3.65983413866e-05
{1: 2.06269037087e-05, 2: 2.18185183654e-05, 3: 2.42676815975e-05, 4: 3.65983413866e-05}


In [5]:
shp = r"C:\Projects\rd\planet\proj_cc_sites.shp"
polys = getGeometries_planet(shp)

In [6]:
ms_patches = [getImagePatch_planet(im, polys[0]) for im in all_ms_ims]
sr_patches = [getImagePatch_planet(im, polys[0]) for im in all_ms_ims]
qa_patches = [getImagePatch_planet(im, polys[0]) for im in all_ms_ims]
coeffs = [getCorrCoefs_planet(xml) for xml in all_xml]
dates = [getDates_planet(xml) for xml in all_xml]

In [9]:
#a = [i for i,j in enumerate(samp) if type(j) is not np.ma.core.MaskedArray]

In [10]:
# extract the mean values per band. however, we may be getting nodata (0) pixels influencing the mean.
# may need to break this out into separate steps
xml_inds = [i for i,j in enumerate(ms_patches) if type(j) is np.ma.core.MaskedArray]

ms_temp = [np.ma.masked_equal(arr, 0.0) for arr in ms_patches if type(arr) is np.ma.core.MaskedArray]
ms_means = [arr.mean(axis=1).mean(axis=1) for arr in ms_temp]

sr_temp = [np.ma.masked_equal(arr, 0.0) for arr in sr_patches if type(arr) is np.ma.core.MaskedArray]
sr_means = [arr.mean(axis=1).mean(axis=1) for arr in sr_temp]
ms_means[0].shape

## this code does not take into account the 0 values
# ms_means = [arr.mean(axis=1).mean(axis=1) for arr in ms_patches if type(arr) is np.ma.core.MaskedArray]
# sr_means = [arr.mean(axis=1).mean(axis=1) for arr in sr_patches if type(arr) is np.ma.core.MaskedArray]
# ms_means[0].shape

IndexError: list index out of range

In [11]:
# extract the band means per patch
ms_band1_means = [b[0] for b in ms_means]
ms_band2_means = [b[1] for b in ms_means]
ms_band3_means = [b[2] for b in ms_means]
ms_band4_means = [b[3] for b in ms_means]

sr_scale = 10000
sr_band1_means = [b[0]/sr_scale for b in sr_means]
sr_band2_means = [b[1]/sr_scale for b in sr_means]
sr_band3_means = [b[2]/sr_scale for b in sr_means]
sr_band4_means = [b[3]/sr_scale for b in sr_means]

In [12]:
# put the dates in a usable list corresponding to only overlapping images
date_arr = [dates[i] for i in xml_inds]

# put the correction coefficients into a usable list corresponding to only overlapping images
cf_arr=[]
for i in xml_inds:
    cf_arr.append(np.array([coeffs[i][1], coeffs[i][2], coeffs[i][3], coeffs[i][4]]))
    
cf_arr[0]

IndexError: list index out of range

In [ ]:
# check the calculation
ms_band1_means[0] *cf_arr[0][0]

In [ ]:
# correct the ms images to TOA reflectance
ms_band1_means_cor = [cf[0] * mn for cf,mn in zip(cf_arr, ms_band1_means)]
ms_band2_means_cor = [cf[1] * mn for cf,mn in zip(cf_arr, ms_band2_means)]
ms_band3_means_cor = [cf[2] * mn for cf,mn in zip(cf_arr, ms_band3_means)]
ms_band4_means_cor = [cf[3] * mn for cf,mn in zip(cf_arr, ms_band4_means)]

In [ ]:
# plot the signature (incorporate the date list)
plt.figure(figsize=(30,10))
plt.subplot(1,2,1)
plt.plot(ms_band1_means_cor)
plt.title('corrected MS image')
plt.subplot(1,2,2)
plt.plot(sr_band1_means)
plt.title('SR image scaled by {}'.format(sr_scale))

In [ ]:
np.where(np.array(ms_band1_means)==0.0)

In [ ]:
plt.imshow(misc.bytescale(np.rollaxis(ms_patches[-6][0:3, :, :],0,3)))

In [ ]:
## ones that are all zeros are no data areas!!!


In [ ]:
c = [ms_patches[i] for i in xml_inds]

In [ ]:
print(len(c))
print(len(ms_patches))

In [ ]:
date_arr

In [ ]:
len(all_sr_ims)

In [ ]:
len(all_ms_ims)

In [ ]:
len(all_xml)

In [ ]:
print(all_xml_temp[0])
print(all_sr_ims[0])

In [ ]:
all_xml_temp = [s.replace('MS_SR.tif', 'MS_metadata.xml') for s in all_sr_ims]

In [ ]:
len(all_xml_temp)